In [1]:
# Import the required libraries and dependencies
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import requests
import json
import csv
from six.moves import urllib

# Load .env environment variables
load_dotenv()

True

In [2]:
parsed_url = "https://data.messari.io/api/v2/assets?fields=symbol,slug,metrics/marketcap/rank,metrics/misc_data/sectors,metrics/misc_data/categories,metrics/supply/y_2050,metrics/supply/circulating,metrics/supply/y_2050_issued_percent,metrics/market_data/price_usd,metrics/marketcap/current_marketcap_usd&limit=500"
parsed_bytes = urllib.request.urlopen(parsed_url).read()
json.loads(parsed_bytes)["data"][0]

{'symbol': 'BTC',
 'slug': 'bitcoin',
 'metrics': {'marketcap': {'rank': 1,
   'current_marketcap_usd': 1105072785277.3054},
  'misc_data': {'sectors': ['Currencies'], 'categories': ['Payments']},
  'supply': {'y_2050': 20986335.65,
   'circulating': 18862355.52134574,
   'y_2050_issued_percent': 89.8918714282548},
  'market_data': {'price_usd': 58331.74275431376}}}

In [3]:
symbol = []
slug = []
marketcap_rank = []
marketcap_usd = []
categories = []
sectors = []
price_usd = []
future_supply = []
future_supply_pct = []

for row in json.loads(parsed_bytes)["data"]:
    symbol.append(row["symbol"])
    slug.append(row["slug"])
    marketcap_rank.append(row["metrics"]["marketcap"]["rank"])
    marketcap_usd.append(row["metrics"]["marketcap"]["current_marketcap_usd"])
    categories.append(row["metrics"]["misc_data"]["categories"])
    sectors.append(row["metrics"]["misc_data"]["sectors"])
    price_usd.append(row["metrics"]["market_data"]["price_usd"])
    future_supply.append(row["metrics"]["supply"]["y_2050"])
    future_supply_pct.append(row["metrics"]["supply"]["y_2050_issued_percent"])

In [4]:
crypto = {"symbol": symbol, "name": slug, "marketcap rank": marketcap_rank, "marketcap usd": marketcap_usd, "categories": categories, "sectors": sectors, "price usd": price_usd, "2050 supply": future_supply, "2050 supply percent": future_supply_pct}

In [5]:
crypto_df = pd.DataFrame(crypto)
crypto_df = crypto_df.dropna()

In [6]:
crypto_df = crypto_df[crypto_df.notnull()]

In [7]:
crypto_df

,symbol,name,marketcap rank,marketcap usd,categories,sectors,price usd,2050 supply,2050 supply percent
0,BTC,bitcoin,1,1.105073e+12,[Payments],[Currencies],58331.742754,2.098634e+07,89.891871
1,ETH,ethereum,2,4.629237e+11,[Infrastructure],[Smart Contract Platforms],3924.924267,1.351356e+08,85.866543
3,BNB,binance-coin,4,7.542790e+10,[Financial],[Centralized Exchanges],448.758545,1.000000e+08,108.345641
5,ADA,cardano,6,6.436781e+10,[Infrastructure],[Smart Contract Platforms],1.914611,4.192222e+10,76.172249
6,SOL,solana,7,5.578796e+10,[Infrastructure],[Smart Contract Platforms],183.217055,9.450229e+08,52.698419
...,...,...,...,...,...,...,...,...,...
463,BRD,bread,464,1.877480e+07,[Financial],[Wallet],0.211585,8.886272e+07,100.000000
468,NXT,nxt,469,1.811017e+07,[Infrastructure],[Smart Contract Platforms],0.018128,9.989999e+08,100.000000
471,PAI,project-pai,472,1.753203e+07,[Services],[AI],0.011161,2.097694e+09,85.186971
486,ABT,arcblock,487,1.543739e+07,[Infrastructure],[Application Development],0.156619,1.860000e+08,100.000000


In [8]:
crypto_df["vlad club cost"] = (crypto_df["2050 supply"] / crypto_df["2050 supply percent"]) * .0001 * crypto_df["price usd"]
crypto_df["categories"] = crypto_df["categories"].explode()
crypto_df["sectors"] = crypto_df["sectors"].explode()

In [9]:
crypto_df.head()

,symbol,name,marketcap rank,marketcap usd,categories,sectors,price usd,2050 supply,2050 supply percent,vlad club cost
0,BTC,bitcoin,1,1.105073e+12,Payments,Currencies,58331.742754,2.098634e+07,89.891871,1.361825e+06
1,ETH,ethereum,2,4.629237e+11,Infrastructure,Smart Contract Platforms,3924.924267,1.351356e+08,85.866543,6.176994e+05
3,BNB,binance-coin,4,7.542790e+10,Financial,Centralized Exchanges,448.758545,1.000000e+08,108.345641,4.141916e+04
5,ADA,cardano,6,6.436781e+10,Infrastructure,Smart Contract Platforms,1.914611,4.192222e+10,76.172249,1.053727e+05
6,SOL,solana,7,5.578796e+10,Infrastructure,Smart Contract Platforms,183.217055,9.450229e+08,52.698419,3.285570e+05


In [13]:
crypto_df = crypto_df.drop(columns=["2050 supply", "2050 supply percent"])

In [14]:
crypto_df["marketcap usd"] = crypto_df["marketcap usd"]

In [15]:
crypto_df

,symbol,name,marketcap rank,marketcap usd,categories,sectors,price usd,vlad club cost
0,BTC,bitcoin,1,1.105073e+12,Payments,Currencies,58331.742754,1.361825e+06
1,ETH,ethereum,2,4.629237e+11,Infrastructure,Smart Contract Platforms,3924.924267,6.176994e+05
3,BNB,binance-coin,4,7.542790e+10,Financial,Centralized Exchanges,448.758545,4.141916e+04
5,ADA,cardano,6,6.436781e+10,Infrastructure,Smart Contract Platforms,1.914611,1.053727e+05
6,SOL,solana,7,5.578796e+10,Infrastructure,Smart Contract Platforms,183.217055,3.285570e+05
...,...,...,...,...,...,...,...,...
463,BRD,bread,464,1.877480e+07,Financial,Wallet,0.211585,1.880204e+01
468,NXT,nxt,469,1.811017e+07,Infrastructure,Smart Contract Platforms,0.018128,1.811017e+01
471,PAI,project-pai,472,1.753203e+07,Services,AI,0.011161,2.748352e+01
486,ABT,arcblock,487,1.543739e+07,Infrastructure,Application Development,0.156619,2.913105e+01
